In [6]:
import pandas as pd
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [7]:
news = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [165]:
news = news.dropna(subset=['content', 'source'])
news[:5]

,id,author,source,content,feature,title,url,xinhua
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm,0
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm,0
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm,0
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....,1
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...,0


In [166]:
news = news.sample(5000)

In [167]:
news['xinhua'] = np.where(news['source'].str.contains('新华'), 1, 0)
news[:5]

,id,author,source,content,feature,title,url,xinhua
38619,50993,崔畅（河南）,新华社,新华社照片，三门峡（河南），2017年4月23日\n读书日寻找读书者\n4月23日，在河南省...,"{""type"":""生活方式与休闲"",""site"":""新华社"",""url"":""http://h...",#（社会）（4）读书日寻找读书者,http://home.xinhua-news.com/gdsdetailxhsnew/19...,1
70242,19370,IAN LANGSDON,新华社,新华社照片，外代，2017年5月22日\n（外代二线）影片《每分钟120击》戛纳首映\n5月...,"{""type"":""电视、广播、电影"",""site"":""新华社"",""url"":""http://...",[3]（外代二线）影片《每分钟120击》戛纳首映,http://home.xinhua-news.com/gdsdetailxhsnew/21...,1
38316,51296,韩瑜庆,新华社,新华社照片，常州（江苏），2017年4月23日\n（体育）（3）羽毛球——大师赛：川上纱惠奈...,"{""type"":""羽毛球"",""site"":""新华社"",""url"":""http://home....",（体育）（3）羽毛球——大师赛：川上纱惠奈获亚军,http://home.xinhua-news.com/gdsdetailxhsnew/19...,1
34396,55216,杨舒怡,新华社,\n\n\n 冲绳美军普天间基地。（新华/法新）\n\n新华社北京4月20日新媒体专电（记...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",普天间基地曝污染 美方拒绝日本调查,http://home.xinhua-news.com/gdsdetailxhsnew/19...,1
42673,46939,NaN,新华社,1961年4月25日，中国与老挝正式建交，56年来，在双方共同努力下，中老全面战略合作伙伴关...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",“一带一路”全球行：中老铁路相联东盟,http://home.xinhua-news.com/gdsdetailxhsnew/19...,1


In [168]:
np.where(news['source'].str.contains('新华'), 1, 0)

array([1, 1, 1, ..., 1, 1, 1])

In [171]:
len(news[news['xinhua']==0])

497

In [172]:
sample = news['content'].tolist()

In [174]:
a = ''.join(re.findall(re.compile('\w|\d'), sample[0]))
b = ' '.join(jieba.cut(a))
b

'新华社 照片 三门峡 河南 2017 年 4 月 23 日 n 读书 日 寻找 读书 者 n4 月 23 日 在 河南省 义马市 新区 办事处 千秋 村 95 岁 的 魏祖鹤 老人 右 与 59 岁 的 儿子 一同 阅览 书籍 古文观止 n 当日 是 世界 读书 日 n 新华社 发崔畅 摄'

In [175]:
def cut(content):
    def remove_extra_char(content):
        content =  [''.join(re.findall(re.compile('\w|\d'), con)) for con in content]
        return content
    
    content = remove_extra_char(content)
    
    jieba.enable_parallel(4)
    
    return [' '.join(jieba.cut(con)) for con in content]

In [176]:
tokens = cut(sample)
tokens[:5]

['新华社 照片 三门峡 河南 2017 年 4 月 23 日 n 读书 日 寻找 读书 者 n4 月 23 日 在 河南省 义马市 新区 办事处 千秋 村 95 岁 的 魏祖鹤 老人 右 与 59 岁 的 儿子 一同 阅览 书籍 古文观止 n 当日 是 世界 读书 日 n 新华社 发崔畅 摄',
 '新华社 照片 外代 2017 年 5 月 22 日 n 外代 二线 影片 每分钟 120 击 戛纳 首映 n5 月 20 日 在 法国 戛纳 美国 模特 肯德尔 詹娜 出席 影片 每分钟 120 击 首映式 n 金棕榈奖 入围 影片 每分钟 120 击 当日 在 第 70 届 戛纳 电影节 首映 n 新华社 欧新 n',
 '新华社 照片 常州 江苏 2017 年 4 月 23 日 n 体育 3 羽毛球 大师赛 川上 纱惠奈获 亚军 n4 月 23 日 日本 选手 川上 纱惠奈 在 比赛 中 回球 n 当日 在 江苏 常州 举行 的 2017 中国羽毛球 大师赛 女单 决赛 中 日本 选手 川上 纱惠奈以 1 比 2 不敌 队友 大堀彩 获得 亚军 n 新华社 记者 韩瑜庆摄 nn',
 'nnn 冲绳 美军 普天间基地 新华 法新 nn 新华社 北京 4 月 20 日新 媒体 专电 记者 杨舒怡据 日本 官员 透露 冲绳 美军 普天间基地 的 地下水 发现 两种 致癌物质 浓度 超标 由于 美方 拒绝 日方 派 人 进行 彻查 眼下 无法 准确 判断 污染物 的 来源 n 不过 这 两种 致癌物 被 用于 制造 灭火剂 航空 润滑油 等 嫌疑 矛头 直指 驻日 美军基地 n 日方 查出 污染 n 今日 俄罗斯电视台 网站 19 日 援引 日本 冲绳县 政府 环保部门 的 一份 声明 报道 根据 该 部门 2016 年 8 月 2017 年 1 月 的 两次 地下水 检测 美军 普天间基地 及 周边地区 的 地下水 中 发现 高浓度 的 全氟 辛基 磺酸 化合物 PFOS 全氟 辛酸 类化合物 PFOA 两种 有毒 物质 n 冲绳县 环保部门 负责 水质 检测 的 官员 宫平吉成 音译 说 在 普天间基地 附近 的 大山 喜友名 两处 区域 的 地下水 均 检测 到 高浓度 的 毒物 引起 日方 警觉 n 日本 自 2010 年 以来 禁止 生产 和 进口 PFOSP

Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-14:
Process ForkPoolWorker-16:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/a

In [177]:
with open('baidu_stop_word.txt', 'r') as f:
    baidu_stop_words = f.read()
baidu_stop_words = baidu_stop_words.split()

In [178]:
vectorizer = TfidfVectorizer(stop_words = baidu_stop_words)
X = vectorizer.fit_transform(tokens)
X.shape

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(5000, 69040)

In [21]:
#vectorizer.vocabulary_

In [179]:
tf_idf_vector = X.toarray()

In [180]:
tf_idf_vector.shape

(5000, 69040)

In [181]:
vectorizer.vocabulary_["这是"]

61986

In [182]:
np.where(tf_idf_vector[0])

(array([  711,   838,  1566,  2024,  2737,  3447,  4503,  5481,  7212,
         7703, 11708, 15424, 16505, 18132, 18650, 26770, 30586, 36612,
        36615, 43413, 43419, 46208, 54164, 59272, 64859, 68330]),)

In [183]:
pca =  PCA(n_components = 100)
#pca_train_x = tf_idf_vector - np.mean(tf_idf_vector)
pca_train_x = pca.fit_transform(tf_idf_vector)
pca_train_x.shape

(5000, 100)

In [185]:
X_train, X_test, y_train, y_test = train_test_split(pca_train_x, y, test_size=0.33, random_state=42)

In [360]:
def performance(y, y_hat):
    print("precision: {}".format(precision_score(y, y_hat)))
    print("recall: {}".format(recall_score(y, y_hat)))
    print("roc_auc: {}".format(roc_auc_score(y, y_hat)))

In [187]:
class KNNCluster:
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y
        
    def run(self, x):
        x = x.reshape(1, -1)
        idx = np.argmax(cosine_similarity(self.train_x, x))
        return self.train_y[idx]
    
    def predict(self, test):
        y_hat = []
        for x in test:
            y = self.run(x)
            y_hat.append(y)
        
        return np.array(y_hat)
    
    def get_cos(self, v1, v2):
        return cosine(v1, v2)

In [188]:
knn = KNNCluster(X_train, y_train)

In [189]:
knn_y_hat = knn.predict(X_test)

In [190]:
performance(y_test, knn_y_hat)

precision: 0.937204591492235
recall: 0.940379403794038
roc_auc: 0.7029483225866742


In [191]:
from sklearn.neighbors import KNeighborsClassifier

In [192]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [193]:
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [194]:
sk_knn_y_hat = neigh.predict(X_test)

In [195]:
performance(y_test, sk_knn_y_hat)

precision: 0.942450914014895
recall: 0.943089430894309
roc_auc: 0.727291841883936


In [196]:
full_X_train, full_X_test, full_y_train, full_y_test = train_test_split(tf_idf_vector, y, test_size=0.33, random_state=42)

## SVM

In [197]:
from sklearn.svm import SVC

In [209]:
clf = SVC(kernel = 'rbf')

In [210]:
clf.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [211]:
svm_y_hat = clf.predict(X_test)

In [212]:
performance(y_test, svm_y_hat)

precision: 0.8945454545454545
recall: 1.0
roc_auc: 0.5


## Bayes

In [321]:
from sklearn.naive_bayes import GaussianNB

In [322]:
clf = GaussianNB()

In [323]:
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [324]:
bayes_y_hat = clf.predict(X_test)

In [325]:
performance(y_test, bayes_y_hat)

precision: 0.9675834970530451
recall: 0.6673441734417345
roc_auc: 0.7388445005139707


In [384]:
from scipy.stats import norm
from sklearn import preprocessing

In [470]:
class BayesCluster:
    def __init__(self, train_x, train_y):
        self.train_x = preprocessing.normalize(train_x)
        self.train_y = train_y
        
        self.prepare()
        
    def prepare(self):
        self.y_true_x_mean = np.mean(self.train_x[self.train_y==1,:], axis=0,)
        self.y_true_x_std = np.std(self.train_x[self.train_y==1,:], axis=0,)
        
        #print(self.y_true_x_mean)
        #print(self.y_true_x_std)
        
        self.y_false_x_mean = np.mean(self.train_x[self.train_y==0,:], axis=0,)
        self.y_false_x_std = np.std(self.train_x[self.train_y==0,:], axis=0,)
        
        #print(self.y_false_x_mean)
        #print(self.y_false_x_std)
    
        self.y_true = len(self.train_x[self.train_y==1]) / len(self.train_x)
        self.y_false = len(self.train_x[self.train_y==0]) / len(self.train_x)

        
    
    def predict(self, test):
        y_hat = []
        p_y_true = 1
        p_y_false = 1
        for vector in test:
            #print(vector)
            for idx, v in enumerate(vector):
                #print(idx)
                p_y_true *= norm.cdf(v, loc = self.y_true_x_mean[idx], scale = self.y_true_x_std[idx])
                p_y_false *= norm.cdf(v, loc = self.y_false_x_mean[idx], scale = self.y_false_x_std[idx])
            
                if p_y_true < 1e-100:
                    p_y_true *= 1e10
                if p_y_false < 1e-100:
                    p_y_false *= 1e10
                #print(p_y_true)
                #print(p_y_false)
            
            p_y_true *= self.y_true
            p_y_false *= self.y_false
            
            #print(p_y_true)
            #print(p_y_false)
            #print('----------------')
            if p_y_true > p_y_false:
                y_hat.append(1)
            else:
                y_hat.append(0)
                
        return np.array(y_hat)

In [471]:
bayes = BayesCluster(X_train, y_train)

In [472]:
bayes_y_hat = bayes.predict(preprocessing.normalize(X_test))
bayes_y_hat

array([1, 1, 0, ..., 0, 0, 1])

In [473]:
performance(y_test, bayes_y_hat)

precision: 0.895397489539749
recall: 0.5799457994579946
roc_auc: 0.5026165778899169
